In [1]:
from constants import PROMPT_MIX_RESPONSE, PROMPT_SANITY_CHECK
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, AutoModelForCausalLM, BitsAndBytesConfig, get_cosine_schedule_with_warmup
from PIL import Image
import pandas as pd
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
from torch.optim import AdamW

c:\Users\Gabriel\anaconda3\envs\visao-computacional\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configurações do Modelo

In [2]:
# verificando se GPU está disponível

print("GPU atual:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma")

GPU atual: NVIDIA GeForce RTX 4060


In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [4]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="cuda:0", quantization_config=quantization_config
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

`torch_dtype` is deprecated! Use `dtype` instead!
Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\Gabriel\anaconda3\envs\visao-computacional\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\Gabriel\anaconda3\envs\visao-computacional\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Gabriel\anaconda3\envs\visao-computacional\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Gabriel\anaconda3\envs\visao-computacional\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc6 in position 8: invalid continuation byte
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.33s/it]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint 

## Generate Answers Functions

In [40]:
def generate_answers_image_only(df: pd.DataFrame, model):

    answers = []
    model_df = df.copy()
    base_image_path = 'images/'

    for i in model_df.index:
        image_infos = model_df.iloc[i]
        image_question = image_infos['question']
        image_path = base_image_path + image_infos['image_path']
        image = Image.open(image_path).convert("RGB")
        prompt_text = image_question + '' + 'Respond only with the final answer, without explanation or full sentences.'

        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {"type": "text", "text": prompt_text},
                ],
            }
        ]

        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = processor(
            text=text,
            images=image,
            return_tensors="pt",
            padding=True,
        ).to(model.device)

        generated_ids = model.generate(**inputs, max_new_tokens=24)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        answer = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        answers.append(answer[0])

        print(f"Resposta gerada para a imagem {i}: {answer[0]}")

    return answers


In [41]:
def generate_answers_text_only(text_col: str, df: pd.DataFrame, model):

    answers = []
    model_df = df.copy()

    for i in model_df.index:
        image_infos = model_df.iloc[i]
        image_description = image_infos[text_col]
        image_question = image_infos['question']

        prompt_text = PROMPT_SANITY_CHECK.format(text=image_description, question=image_question)

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                ],
            }
        ]

        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = processor(
            text=text,
            return_tensors="pt",
            padding=True,
        ).to(model.device)

        generated_ids = model.generate(**inputs, max_new_tokens=24)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        answer = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        answers.append(answer[0])

        print(f"Resposta gerada para a imagem {i}: {answer[0]}")

    return answers


In [42]:
def generate_answers_mix(text_col: str, df: pd.DataFrame, model):

    answers = []
    model_df = df.copy()
    base_image_path = 'images/'

    for i in model_df.index:
        image_infos = model_df.iloc[i]
        image_question = image_infos['question']
        image_description = image_infos[text_col]
        image_path = base_image_path + image_infos['image_path']
        image = Image.open(image_path).convert("RGB")
        prompt_text = PROMPT_MIX_RESPONSE.format(text_information=image_description, question=image_question)

        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {"type": "text", "text": prompt_text},
                ],
            }
        ]

        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = processor(
            text=text,
            images=image,
            return_tensors="pt",
            padding=True,
        ).to(model.device)

        generated_ids = model.generate(**inputs, max_new_tokens=24)
        generated_ids_trimmed = [
            out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        answer = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )

        answers.append(answer[0])

        print(f"Resposta gerada para a imagem {i}: {answer[0]}")

    return answers

## Generating Answers

In [ ]:
df = pd.read_csv('data/final_data_descriptions.csv')

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,image_path,image_id,question,answer,Tm,Tc,Ans,Ti
0,0,0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red,"Several days after the low dissipated , the r..."
1,1,1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,red,Kawaguchi 's Center Body of troops was planni...
2,2,2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red,"In the centre , the main attack along the Bui..."
3,3,3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red,Mount Elbert was named by miners in honor of ...
4,4,4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,red,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...,...,...,...,...
991,991,991,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no,The documentary film Tim Richmond : To The Li...
992,992,992,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics.",Kevin Spacey as David <unk> \r\n
993,993,993,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,blue,The Island Def Jam rapper Big K.R.I.T. was bo...
994,994,994,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red,"A total of 2 @,@ 000 people attended Slammive..."


In [ ]:
qwen_df = df[['image_path', 'question', 'answer', 'Tm', 'Tc', 'Ti']]

In [ ]:
qwen_df

,image_path,question,answer,Tm,Tc,Ti
0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r..."
1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...
2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui..."
3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...
4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...
991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...
992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \r\n
993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...
994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive..."


In [ ]:
qwen_tm_responses = generate_answers_text_only('Tm', qwen_df, model)

Resposta gerada para a imagem 0: Blue
Resposta gerada para a imagem 1: SV-6260
Resposta gerada para a imagem 2: Yes
Resposta gerada para a imagem 3: Blue
Resposta gerada para a imagem 4: no
Resposta gerada para a imagem 5: Looking outside
Resposta gerada para a imagem 6: Yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: Baseball
Resposta gerada para a imagem 9: Corn
Resposta gerada para a imagem 10: Turn on the cold water by opening the faucet.
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: Red
Resposta gerada para a imagem 13: Cupcake
Resposta gerada para a imagem 14: Yes
Resposta gerada para a imagem 15: Yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: No
Resposta gerada para a imagem 18: Umpires
Resposta gerada para a imagem 19: Overcast
Resposta gerada para a imagem 20: Yes
Resposta gerada para a imagem 21: on top of red laptop
Resposta gerada para a imagem 22: orange
Resposta gerada para a imagem 23: 

In [ ]:
qwen_ti_responses = generate_answers_text_only('Ti', qwen_df, model)

Resposta gerada para a imagem 0: Black
Resposta gerada para a imagem 1: <unk> gata
Resposta gerada para a imagem 2: No
Resposta gerada para a imagem 3: White
Resposta gerada para a imagem 4: No
Resposta gerada para a imagem 5: The woman in the room is sitting.
Resposta gerada para a imagem 6: Black
Resposta gerada para a imagem 7: 0
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: Cotton
Resposta gerada para a imagem 10: Turn on the cold water
Resposta gerada para a imagem 11: The Million Dollar Hotel
Resposta gerada para a imagem 12: Black
Resposta gerada para a imagem 13: The brand of wine is not mentioned in the given text.
Resposta gerada para a imagem 14: No
Resposta gerada para a imagem 15: No
Resposta gerada para a imagem 16: stage
Resposta gerada para a imagem 17: No
Resposta gerada para a imagem 18: Churchill
Resposta gerada para a imagem 19: Overcast
Resposta gerada para a imagem 20: No
Resposta gerada para a imagem 21: Highland Park
Resposta gerada 

In [ ]:
qwen_tc_responses = generate_answers_text_only('Tc', qwen_df, model)

Resposta gerada para a imagem 0: Red
Resposta gerada para a imagem 1: AB-1234
Resposta gerada para a imagem 2: Yes
Resposta gerada para a imagem 3: Red
Resposta gerada para a imagem 4: no
Resposta gerada para a imagem 5: reading
Resposta gerada para a imagem 6: Red
Resposta gerada para a imagem 7: 3
Resposta gerada para a imagem 8: soccer
Resposta gerada para a imagem 9: lemons
Resposta gerada para a imagem 10: Turn on the cold water by opening the faucet.
Resposta gerada para a imagem 11: chair
Resposta gerada para a imagem 12: Green
Resposta gerada para a imagem 13: Sunset
Resposta gerada para a imagem 14: No
Resposta gerada para a imagem 15: No
Resposta gerada para a imagem 16: metal chair
Resposta gerada para a imagem 17: Yes
Resposta gerada para a imagem 18: Coaches
Resposta gerada para a imagem 19: Sunny
Resposta gerada para a imagem 20: No
Resposta gerada para a imagem 21: on top of blue laptop
Resposta gerada para a imagem 22: Blue
Resposta gerada para a imagem 23: Spinach
Resp

In [ ]:
qwen_base_response = generate_answers_image_only(qwen_df, model)

Resposta gerada para a imagem 0: white
Resposta gerada para a imagem 1: 6260
Resposta gerada para a imagem 2: no
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: yes
Resposta gerada para a imagem 5: sitting
Resposta gerada para a imagem 6: yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: corn
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: red
Resposta gerada para a imagem 13: cupcake
Resposta gerada para a imagem 14: yes
Resposta gerada para a imagem 15: yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: no
Resposta gerada para a imagem 18: umpire
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: red
Resposta gerada para a imagem 23: basil
Resposta gerada para a imagem 24: E-820 CR MS 815 AJ
Respos

In [ ]:
qwen_mix_ti_responses = generate_answers_mix('Ti',qwen_df, model)

Resposta gerada para a imagem 0: black
Resposta gerada para a imagem 1: SV-6260
Resposta gerada para a imagem 2: No
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: Yes
Resposta gerada para a imagem 5: The woman is standing.
Resposta gerada para a imagem 6: Yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: Corn
Resposta gerada para a imagem 10: Turn faucet
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: Red
Resposta gerada para a imagem 13: Cupcake
Resposta gerada para a imagem 14: Yes
Resposta gerada para a imagem 15: Yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: No
Resposta gerada para a imagem 18: Umpire
Resposta gerada para a imagem 19: Overcast
Resposta gerada para a imagem 20: Yes
Resposta gerada para a imagem 21: Asus
Resposta gerada para a imagem 22: red
Resposta gerada para a imagem 23: basil
Resposta gerada para a imagem 24: E-820 

In [ ]:
qwen_mix_tc_responses = generate_answers_mix('Tc',qwen_df, model)

Resposta gerada para a imagem 0: red
Resposta gerada para a imagem 1: SV-6260
Resposta gerada para a imagem 2: Yes
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: Yes
Resposta gerada para a imagem 5: The woman is standing.
Resposta gerada para a imagem 6: Red
Resposta gerada para a imagem 7: 2
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: lemons
Resposta gerada para a imagem 10: Turn the faucet
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: Red
Resposta gerada para a imagem 13: Sunset
Resposta gerada para a imagem 14: Yes
Resposta gerada para a imagem 15: Yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: No
Resposta gerada para a imagem 18: Coaches
Resposta gerada para a imagem 19: Overcast
Resposta gerada para a imagem 20: Yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: red
Resposta gerada para a imagem 23: spinach
Resposta gerada para a imagem 24

In [ ]:
qwen_mix_tm_responses = generate_answers_mix('Tm',qwen_df, model)

Resposta gerada para a imagem 0: blue
Resposta gerada para a imagem 1: SV-6260
Resposta gerada para a imagem 2: Yes
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: no
Resposta gerada para a imagem 5: Standing
Resposta gerada para a imagem 6: Yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: Baseball
Resposta gerada para a imagem 9: corn
Resposta gerada para a imagem 10: Turn faucet
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: Red
Resposta gerada para a imagem 13: Cupcake
Resposta gerada para a imagem 14: Yes
Resposta gerada para a imagem 15: Yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: No
Resposta gerada para a imagem 18: Umpires
Resposta gerada para a imagem 19: Overcast
Resposta gerada para a imagem 20: Yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: orange
Resposta gerada para a imagem 23: basil
Resposta gerada para a imagem 24: E-820 CR MS 8

In [ ]:
qwen_df['Tc_responses'] = pd.Series(qwen_tc_responses)
qwen_df['Tm_responses'] = pd.Series(qwen_tm_responses)
qwen_df['Ti_responses'] = pd.Series(qwen_ti_responses)

qwen_df['base_responses'] = pd.Series(qwen_base_response)
qwen_df['mix_tc_responses'] = pd.Series(qwen_mix_tc_responses)
qwen_df['mix_tm_responses'] = pd.Series(qwen_mix_tm_responses)
qwen_df['mix_ti_responses'] = pd.Series(qwen_mix_ti_responses)

In [ ]:
qwen_df.to_csv('data/qwen_2b_responses.csv')

## Fine-Tuning

In [5]:
fst_df = pd.read_csv('data/final_df_ft.csv')

In [6]:
fst_df

,Unnamed: 0,image_path,image_id,question,answer,type,Tm,Tc,Ti
0,0,COCO_val2014_000000014248.jpg,14248,What is laying on the bed?,girl,text only,NaN,NaN,NaN
1,1,COCO_val2014_000000014257.jpg,14257,Are the two people friends?,yes,text only,NaN,NaN,NaN
2,2,COCO_val2014_000000014265.jpg,14265,Is the man carrying a bag?,yes,text only,NaN,NaN,NaN
3,3,COCO_val2014_000000014271.jpg,14271,What kind is the tree?,oak,text only,NaN,NaN,NaN
4,4,COCO_val2014_000000014276.jpg,14276,What is the wall made of?,trees,text only,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
995,995,COCO_val2014_000000027932.jpg,27932,Who does the equipment belong too?,baseball player,irrelevant,NaN,NaN,Liaison planes rose from the division strip e...
996,996,COCO_val2014_000000027935.jpg,27935,How many flowers are in each bucket?,lot,irrelevant,NaN,NaN,Gorilla press slam \n
997,997,COCO_val2014_000000027946.jpg,27946,Urban or suburban?,urban,irrelevant,NaN,NaN,"On 6 November , Édouard Adolphe Mortier comma..."
998,998,COCO_val2014_000000027969.jpg,27969,What is behind the woman?,parking lot,irrelevant,NaN,NaN,Manila is politically divided into 896 barang...


In [7]:
fst_df = fst_df.fillna('')

In [11]:
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    bias="none"
)

model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

trainable params: 4,358,144 || all params: 2,213,343,744 || trainable%: 0.1969


In [12]:
def convert_dataset_to_tuning(fst_df: pd.DataFrame):

    infos_fst = []

    for i in fst_df.index:
        
        image_infos = fst_df.iloc[i]
        image_question = image_infos['question']
        image_answer = image_infos['answer']

        type = image_infos['type']

        text_match = image_infos['Tm']
        text_corrupted = image_infos['Tc']
        text_irrelevant = image_infos['Ti']

        image_path = 'images_fine_tuning/' + image_infos['image_path']
        image = Image.open(image_path).convert("RGB")

        info_fst = {"image": image, "question": image_question, "answer": image_answer, "type": type, "text_match": text_match, "text_corrupted": text_corrupted, "text_irrelevant": text_irrelevant}
        infos_fst.append(info_fst)

    return infos_fst

In [25]:
def collate(batch):
    images = [item["image"] for item in batch]
    questions = [item["question"] for item in batch]
    answers = [item["answer"] for item in batch]
    text_match = [item["text_match"] for item in batch]
    text_corrupted = [item["text_corrupted"] for item in batch]
    text_irrelevant = [item["text_irrelevant"] for item in batch]
    type = [item["type"] for item in batch]

    prompt = ''
    for i in range(len(batch)): # como o batch será de 1 única imagem, não sobreescreverá os valores. passando assim apenas pelo formato do loader

        if text_match[i]:
            prompt = questions[i] + ' ' + text_match[i]
        elif text_corrupted[i]:
            prompt = questions[i] + ' ' + text_corrupted[i]
        elif text_irrelevant[i]:
            prompt = questions[i] + ' ' + text_irrelevant[i]
        else: 
            prompt = questions[i]

        images = images[i]
        answers = answers[i]

    if type == 'text only':

        messages = [
            [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            },

            {
                    "role": "assistant", 
                    "content": answers
            }   
            ]      
        ]

    else:
        messages = [
            [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                    },
                    {"type": "text", "text": prompt},
                ],
            },

            {
                    "role": "assistant", 
                    "content": answers
            }   
            ]     
        ]

    texts = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
    ).to(model.device)

    generated_ids = model.generate(**inputs, max_new_tokens=24)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    answer = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    enc = processor(
        text=texts,
        images=images,
        padding="longest",
        return_tensors="pt"
    )

    labels = enc.input_ids.clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100

    return {
        "input_ids": enc.input_ids,
        "attention_mask": enc.attention_mask,
        "pixel_values": enc.pixel_values if "pixel_values" in enc else None,
        "image_grid_thw": enc.image_grid_thw if "image_grid_thw" in enc else None,
        "labels": labels,
    }

In [14]:
dataset_fst = convert_dataset_to_tuning(fst_df)

In [27]:
loader = DataLoader(dataset_fst, batch_size=1, shuffle=True, collate_fn=collate)

In [28]:
lr = 1e-4
epochs = 3

optimizer = AdamW(model.parameters(), lr=lr)

num_steps = len(loader) * epochs
warmup_steps = int(0.1 * num_steps)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=num_steps
)

In [32]:
model.save_pretrained("modelo_tunado")

In [31]:
from tqdm import tqdm

model.train()
step_count = 0

for epoch in range(epochs):
    for batch in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        
        out = model(**batch)
        loss = out.loss

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        step_count += 1

        if step_count % 100 == 0:
            print(f"Step {step_count}/{num_steps} | Loss = {loss.item():.4f}")

Epoch 1/3:  10%|█         | 100/1000 [04:48<38:03,  2.54s/it] 

Step 100/3000 | Loss = 7.7572


Epoch 1/3:  20%|██        | 200/1000 [08:53<29:46,  2.23s/it]  

Step 200/3000 | Loss = 7.5067


Epoch 1/3:  30%|███       | 300/1000 [11:36<17:17,  1.48s/it]

...

Step 2800/3000 | Loss = 7.2327


Epoch 3/3:  90%|█████████ | 900/1000 [22:16<02:22,  1.43s/it]

Step 2900/3000 | Loss = 7.0076


Epoch 3/3: 100%|██████████| 1000/1000 [24:35<00:00,  1.48s/it]

Step 3000/3000 | Loss = 7.2538


In [33]:
model.save_pretrained("qwen2vl-2b-vqa-lora")

## Generating Answers Tuning

In [46]:
df = pd.read_csv('data/final_data_descriptions.csv')

In [47]:
df

,Unnamed: 0.1,Unnamed: 0,image_path,image_id,question,answer,Tm,Tc,Ans,Ti
0,0,0,COCO_val2014_000000000042.jpg,42,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,red,"Several days after the low dissipated , the r..."
1,1,1,COCO_val2014_000000000073.jpg,73,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,red,Kawaguchi 's Center Body of troops was planni...
2,2,2,COCO_val2014_000000000074.jpg,74,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,red,"In the centre , the main attack along the Bui..."
3,3,3,COCO_val2014_000000000133.jpg,133,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,red,Mount Elbert was named by miners in honor of ...
4,4,4,COCO_val2014_000000000136.jpg,136,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,red,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...,...,...,...,...
991,991,991,COCO_val2014_000000014135.jpg,14135,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,no,The documentary film Tim Richmond : To The Li...
992,992,992,COCO_val2014_000000014151.jpg,14151,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,"No, this is not at the Olympics.",Kevin Spacey as David <unk> \r\n
993,993,993,COCO_val2014_000000014167.jpg,14167,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,blue,The Island Def Jam rapper Big K.R.I.T. was bo...
994,994,994,COCO_val2014_000000014175.jpg,14175,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,red,"A total of 2 @,@ 000 people attended Slammive..."


In [48]:
qwen_tuning_df = df[['image_path', 'question', 'answer', 'Tm', 'Tc', 'Ti']]

In [49]:
qwen_tuning_df

,image_path,question,answer,Tm,Tc,Ti
0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r..."
1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...
2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui..."
3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...
4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve..."
...,...,...,...,...,...,...
991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...
992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \r\n
993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...
994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive..."


In [50]:
qwen_tuning_tm_responses = generate_answers_text_only('Tm', qwen_tuning_df, model)

Resposta gerada para a imagem 0: blue
Resposta gerada para a imagem 1: SV-6260
Resposta gerada para a imagem 2: yes
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: yes
Resposta gerada para a imagem 5: looking outside
Resposta gerada para a imagem 6: yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: corn
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: red
Resposta gerada para a imagem 13: Cupcake
Resposta gerada para a imagem 14: yes
Resposta gerada para a imagem 15: yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: no
Resposta gerada para a imagem 18: umpires
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: yes
Resposta gerada para a imagem 21: on top of white laptop
Resposta gerada para a imagem 22: orange
Resposta gerada para a imagem 23: basil
Resposta gerada para a imagem 

In [51]:
qwen_tuning_tc_responses = generate_answers_text_only('Tc', qwen_tuning_df, model)

Resposta gerada para a imagem 0: red
Resposta gerada para a imagem 1: AB-1234
Resposta gerada para a imagem 2: yes
Resposta gerada para a imagem 3: red
Resposta gerada para a imagem 4: no
Resposta gerada para a imagem 5: reading
Resposta gerada para a imagem 6: red
Resposta gerada para a imagem 7: 3
Resposta gerada para a imagem 8: soccer
Resposta gerada para a imagem 9: lemons
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: chair
Resposta gerada para a imagem 12: green
Resposta gerada para a imagem 13: Sunset
Resposta gerada para a imagem 14: no
Resposta gerada para a imagem 15: no
Resposta gerada para a imagem 16: chair
Resposta gerada para a imagem 17: yes
Resposta gerada para a imagem 18: coaches
Resposta gerada para a imagem 19: sunny
Resposta gerada para a imagem 20: no
Resposta gerada para a imagem 21: on top of green laptop
Resposta gerada para a imagem 22: blue
Resposta gerada para a imagem 23: spinach
Resposta gerada para a imagem 24: MS 123 XY
Resp

In [52]:
qwen_tuning_ti_responses = generate_answers_text_only('Ti', qwen_tuning_df, model)

Resposta gerada para a imagem 0: black
Resposta gerada para a imagem 1: 12 33 44
Resposta gerada para a imagem 2: no
Resposta gerada para a imagem 3: white
Resposta gerada para a imagem 4: no
Resposta gerada para a imagem 5: sitting
Resposta gerada para a imagem 6: black
Resposta gerada para a imagem 7: 100
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: banana
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: chair
Resposta gerada para a imagem 12: black
Resposta gerada para a imagem 13: Chateau
Resposta gerada para a imagem 14: no
Resposta gerada para a imagem 15: no
Resposta gerada para a imagem 16: chairs
Resposta gerada para a imagem 17: no
Resposta gerada para a imagem 18: Churchill
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: no
Resposta gerada para a imagem 21: Highland Park
Resposta gerada para a imagem 22: black
Resposta gerada para a imagem 23: leaves
Resposta gerada para a imagem 24: BANG!
R

In [53]:
qwen_tuning_base_response = generate_answers_image_only(qwen_tuning_df, model)

Resposta gerada para a imagem 0: white
Resposta gerada para a imagem 1: 6260
Resposta gerada para a imagem 2: no
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: yes
Resposta gerada para a imagem 5: eating
Resposta gerada para a imagem 6: yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: corn
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: floor
Resposta gerada para a imagem 12: red
Resposta gerada para a imagem 13: cupcake
Resposta gerada para a imagem 14: yes
Resposta gerada para a imagem 15: yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: no
Resposta gerada para a imagem 18: umpire
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: red
Resposta gerada para a imagem 23: basil
Resposta gerada para a imagem 24: e 820 cr
Resposta gerada p

In [54]:
qwen_tuning_mix_ti_responses = generate_answers_mix('Ti',qwen_tuning_df, model)

Resposta gerada para a imagem 0: black
Resposta gerada para a imagem 1: SV 6260
Resposta gerada para a imagem 2: yes
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: yes
Resposta gerada para a imagem 5: sitting
Resposta gerada para a imagem 6: yellow
Resposta gerada para a imagem 7: 2
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: corn
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: red
Resposta gerada para a imagem 13: Cupcake
Resposta gerada para a imagem 14: yes
Resposta gerada para a imagem 15: yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: no
Resposta gerada para a imagem 18: baseball players
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: red
Resposta gerada para a imagem 23: spinach
Resposta gerada para a imagem 24: e-820 cr


In [55]:
qwen_tuning_mix_tc_responses = generate_answers_mix('Tc',qwen_tuning_df, model)

Resposta gerada para a imagem 0: red
Resposta gerada para a imagem 1: AB-1234
Resposta gerada para a imagem 2: yes
Resposta gerada para a imagem 3: red
Resposta gerada para a imagem 4: yes
Resposta gerada para a imagem 5: reading
Resposta gerada para a imagem 6: red
Resposta gerada para a imagem 7: 3
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: lemons
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: chair
Resposta gerada para a imagem 12: red
Resposta gerada para a imagem 13: Sunset
Resposta gerada para a imagem 14: yes
Resposta gerada para a imagem 15: yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: yes
Resposta gerada para a imagem 18: baseball coaches
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: red
Resposta gerada para a imagem 23: spinach
Resposta gerada para a imagem 24: e 820 cr
Resp

In [56]:
qwen_tuning_mix_tm_responses = generate_answers_mix('Tm',qwen_tuning_df, model)

Resposta gerada para a imagem 0: black
Resposta gerada para a imagem 1: SV-6260
Resposta gerada para a imagem 2: yes
Resposta gerada para a imagem 3: blue
Resposta gerada para a imagem 4: yes
Resposta gerada para a imagem 5: standing
Resposta gerada para a imagem 6: yellow
Resposta gerada para a imagem 7: 1
Resposta gerada para a imagem 8: baseball
Resposta gerada para a imagem 9: corn
Resposta gerada para a imagem 10: handle
Resposta gerada para a imagem 11: couch
Resposta gerada para a imagem 12: red
Resposta gerada para a imagem 13: Cupcake
Resposta gerada para a imagem 14: yes
Resposta gerada para a imagem 15: yes
Resposta gerada para a imagem 16: bench
Resposta gerada para a imagem 17: no
Resposta gerada para a imagem 18: umpires
Resposta gerada para a imagem 19: overcast
Resposta gerada para a imagem 20: yes
Resposta gerada para a imagem 21: on table
Resposta gerada para a imagem 22: orange
Resposta gerada para a imagem 23: basil
Resposta gerada para a imagem 24: e-820 cr
Respost

In [57]:
qwen_tuning_df['Tc_responses'] = pd.Series(qwen_tuning_tc_responses)
qwen_tuning_df['Tm_responses'] = pd.Series(qwen_tuning_tm_responses)
qwen_tuning_df['Ti_responses'] = pd.Series(qwen_tuning_ti_responses)

qwen_tuning_df['base_responses'] = pd.Series(qwen_tuning_base_response)
qwen_tuning_df['mix_tc_responses'] = pd.Series(qwen_tuning_mix_tc_responses)
qwen_tuning_df['mix_tm_responses'] = pd.Series(qwen_tuning_mix_tm_responses)
qwen_tuning_df['mix_ti_responses'] = pd.Series(qwen_tuning_mix_ti_responses)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_6468\3358555526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qwen_tuning_df['Tc_responses'] = pd.Series(qwen_tuning_tc_responses)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_6468\3358555526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qwen_tuning_df['Tm_responses'] = pd.Series(qwen_tuning_tm_responses)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_6468\3358555526.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [58]:
qwen_tuning_df.to_csv('data/qwen_tuning_2b_responses.csv')

In [59]:
qwen_tuning_df

,image_path,question,answer,Tm,Tc,Ti,Tc_responses,Tm_responses,Ti_responses,base_responses,mix_tc_responses,mix_tm_responses,mix_ti_responses
0,COCO_val2014_000000000042.jpg,What color are the gym shoes?,white,A curly-haired dog is sleeping on a shoe rack ...,A curly-haired dog is sleeping on a shoe rack ...,"Several days after the low dissipated , the r...",red,blue,black,white,red,black,black
1,COCO_val2014_000000000073.jpg,What is the license number?,sv-6260,The motorcycle in the image has a license plat...,The motorcycle in the image has a license plat...,Kawaguchi 's Center Body of troops was planni...,AB-1234,SV-6260,12 33 44,6260,AB-1234,SV-6260,SV 6260
2,COCO_val2014_000000000074.jpg,Does this dog have a collar?,no,The image shows a white dog sleeping on a cobb...,The image shows a white dog sleeping on a cobb...,"In the centre , the main attack along the Bui...",yes,yes,no,no,yes,yes,yes
3,COCO_val2014_000000000133.jpg,What color is lamp?,blue,The image shows a wooden loft bed with a small...,The image shows a wooden loft bed with a small...,Mount Elbert was named by miners in honor of ...,red,blue,white,blue,red,blue,blue
4,COCO_val2014_000000000136.jpg,Is this in a museum?,no,The image shows two giraffes in an indoor zoo ...,The image shows two giraffes in a natural sava...,"Runs west through Jackson , Mississippi , eve...",no,yes,no,yes,yes,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,COCO_val2014_000000014135.jpg,Is it daytime?,yes,The image shows a skateboarder performing a tr...,The image shows a skateboarder performing a tr...,The documentary film Tim Richmond : To The Li...,no,yes,no,yes,no,yes,yes
992,COCO_val2014_000000014151.jpg,Is this at the Olympics?,yes,A ski jumper is in mid-air above a snow-covere...,A ski jumper is in mid-air above a snow-covere...,Kevin Spacey as David <unk> \r\n,no,yes,no,yes,yes,yes,yes
993,COCO_val2014_000000014167.jpg,Is the skateboarder wearing safety gear?,no,The skateboarder is performing a trick down a ...,The skateboarder is performing a trick down a ...,The Island Def Jam rapper Big K.R.I.T. was bo...,yes,no,no,no,yes,no,no
994,COCO_val2014_000000014175.jpg,What is sticking up from the fire hydrant?,nothing,The image shows a street scene with a fire hyd...,The image shows a street scene with a fire hyd...,"A total of 2 @,@ 000 people attended Slammive...",stick,object,nothing,sign,stick,blue object,fire hydrant
